<h1 align="center">
  Nienadzorowana reprezentacja autoenkodery i modele generatywne
</h1>

<h4 align="center">
  11.10.2023
</h4>
<br/>


# Redukcja wymiarowości

**Analiza głównych składowych** (ang. Principal Component Analysis, PCA) jest jedną ze statystycznych metod analizy czynnikowej. Metoda ta jest często używana do zmniejszania rozmiaru zbioru danych statystycznych, poprzez odrzucenie ostatnich czynników (mniej istotnych). 

Głównym celem analizy PCA jest identyfikacja wzorców w danych. Celem PCA jest wykrycie korelacji między zmiennymi. W skrócie, PCA znajduje kierunki maksymalnej wariancji wysoko-wymiarowych danych, a następnie rzutuje dane na mniej-wymiarową podprzestrzeń, która zachowuje najwięcej informacji z oryginalnego zbioru. Więcej informacji na temat *PCA* można znaleźć <a href="https://en.wikipedia.org/wiki/Principal_component_analysis">tutaj</a>.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

X = np.array([[-1, -1, 4], [-2, -1, 6], [-3, -2, 2], [1, 1, -1], [2, 1, 0], [3, 2, 1]])
pca = PCA(n_components=2)
pca.fit_transform(X)

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
    header=None, 
    sep=',')

df.columns=['1', '2', '3', '4', 'class']
df.dropna(how="all", inplace=True) # usuwamy puste rekordy

print(df['class'].unique())
print(df.shape)
df.head()

Podział tabelki na dane i etykiety (klasy).

In [ ]:
X = df.iloc[:,0:4].values
y = df.iloc[:,4].values

print(X[:5, :])
print(y[:5])

Za pomocą histogramów możemy zobaczyć jak te trzy klasy różnią się wzdłuż każdego wymiaru.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

fig = plt.figure(figsize=(8,6))
plt.hist(X[np.where(y == 'Iris-setosa')[0], 0], bins=7, label='Iris-setosa')
plt.hist(X[np.where(y == 'Iris-versicolor')[0], 0], bins=7, label='Iris-versicolor')
plt.hist(X[np.where(y == 'Iris-virginica')[0], 0], bins=7, label='Iris-virginica')
plt.legend(loc='upper right')
plt.show(fig);

fig = plt.figure(figsize=(8,6))
plt.hist(X[np.where(y == 'Iris-setosa')[0], 1], bins=7, label='Iris-setosa')
plt.hist(X[np.where(y == 'Iris-versicolor')[0], 1], bins=7, label='Iris-versicolor')
plt.hist(X[np.where(y == 'Iris-virginica')[0], 1], bins=7, label='Iris-virginica')
plt.legend(loc='upper right')
plt.show(fig);

fig = plt.figure(figsize=(8,6))
plt.hist(X[np.where(y == 'Iris-setosa')[0], 2], bins=7, label='Iris-setosa')
plt.hist(X[np.where(y == 'Iris-versicolor')[0], 2], bins=7, label='Iris-versicolor')
plt.hist(X[np.where(y == 'Iris-virginica')[0], 2], bins=7, label='Iris-virginica')
plt.legend(loc='upper right')
plt.show(fig);

fig = plt.figure(figsize=(8,6))
plt.hist(X[np.where(y == 'Iris-setosa')[0], 3], bins=7, label='Iris-setosa')
plt.hist(X[np.where(y == 'Iris-versicolor')[0], 3], bins=7, label='Iris-versicolor')
plt.hist(X[np.where(y == 'Iris-virginica')[0], 3], bins=7, label='Iris-virginica')
plt.legend(loc='upper right')
plt.show(fig);

Wykonując analizę głównych składowych powinniśmy standaryzować dane, ponieważ PCA wybiera te podprzestrzenie (najbardziej istotne), które maksymalizują wariancję wzdłuż osi. Aby to zrobić możemy posłużyć się poniższym modułem:

In [ ]:
from sklearn.preprocessing import StandardScaler

X_std = StandardScaler().fit_transform(X)
print(X_std[:5, :])

Klasyczne metody PCA obliczają wartości i wektory własne z macierzy kowariancji. Poniżej pokazujemy jak można w prosty sposób policzyć macierz kowariancji.  

In [ ]:
import numpy as np

mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
# print np.cov(X_std.T) # równoważnie
print('Macierz kowariancji: \n%s' %cov_mat)

Następnie obliczamy problem własny dla macierzy kowariancji. Ponieważ jest to macierz symetryczna dlatego też użyjemy metody, która oblicza problem własny dla macierzy symetrycznej (jest szybsza, mniej złożona obliczeniowo niż metoda licząca problem własny dla macierzy niesymetrycznych).

In [ ]:
eig_vals, eig_vecs = np.linalg.eigh(cov_mat)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Czasem zamiast macierzy kowariancji oblicza się macierz korelacji (szczególnie w finansach). Warto zauważyć, że problem własny dla macierzy kowariancji (jeżeli dane wejściowe są standaryzowane) daje takie same wyniki jak dla macierzy korelacji (macierz korelacji może być rozumiana jako znormalizowana macierzy kowariancji).

In [ ]:
corr = np.corrcoef(X_std.T)
print(corr, "\n")

eig_vals, eig_vecs = np.linalg.eigh(corr)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

In [ ]:
corr1 = np.corrcoef(X.T)
print(corr1, "\n")

eig_vals, eig_vecs = np.linalg.eigh(corr1)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Widzimy, że wszystkie trzy podejścia dają te same wektory i wartości własnej. Większość metod PCA (w implementacji) wykorzystuje rozkład SVD (<a href="https://pl.wikipedia.org/wiki/Rozk%C5%82ad_wed%C5%82ug_warto%C5%9Bci_osobliwych">zobacz</a>) w celu poprawy wydajności obliczeń.

In [ ]:
u,s,v = np.linalg.svd(X_std.T)
print(u, '\n')
print(s, '\n')
print(v)

Gdy już mamy wartości i wektory własne, należy je posortować. Sortujemy malejąco, ponieważ będziemy rzutować na podprzestrzenie (opisane przez wektory własne) przechowujące więcej informacji ze zbioru danych.

In [ ]:
# czy wektory są znormalizowane
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Ok!')

# lista krotek (eigenvalue, eigenvector)
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) 
             for i in range(len(eig_vals))]

# Sortowanie (eigenvalue, eigenvector)
eig_pairs.sort()
eig_pairs.reverse()

print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

Możemy już zmniejszyć wymiar 4-wymiarowych danych do 2-wymiarowe dane. W tym celu wybieramy dwa wektory własne odpowiadające dwóm największym wartością własnym.

In [ ]:
W = np.hstack((eig_pairs[0][1].reshape(-1,1), eig_pairs[1][1].reshape(-1,1)))
print('Macierz W:\n', W)

In [ ]:
Y = X_std.dot(W)
Y[:, 0] = Y[:, 0] * -1

print(X_std[:6,:], '\n')
print(Y[:6, :])

In [ ]:
colors = ['b', 'c', 'r']
fig = plt.figure(figsize=(8,6))
plt.scatter(Y[np.where(y == 'Iris-setosa')[0], 0], Y[np.where(y == 'Iris-setosa')[0], 1],
            marker='x', color=colors[0], label='Iris-setosa')
plt.scatter(Y[np.where(y == 'Iris-versicolor')[0], 0], Y[np.where(y == 'Iris-versicolor')[0], 1], 
            marker='^', color=colors[1], label='Iris-versicolor')
plt.scatter(Y[np.where(y == 'Iris-virginica')[0], 0], Y[np.where(y == 'Iris-virginica')[0], 1], 
            marker='o', color=colors[2], label='Iris-virginica')
plt.legend(loc='upper right')
plt.show(fig);

Biblioteka <a href="http://scikit-learn.org/stable/index.html">**sklearn**</a> dostarcza nam funkcję <a href="http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html">*PCA*</a>, która wykonuje wszystkie powyższe operacje.

In [ ]:
from sklearn.decomposition import PCA as sklearnPCA


sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(X_std)
print(Y_sklearn[:6, :])

In [ ]:
colors = ['b', 'c', 'r']
fig = plt.figure(figsize=(8,6))
plt.scatter(Y_sklearn[np.where(y == 'Iris-setosa')[0], 0], Y_sklearn[np.where(y == 'Iris-setosa')[0], 1],
            marker='x', color=colors[0], label='Iris-setosa')
plt.scatter(Y_sklearn[np.where(y == 'Iris-versicolor')[0], 0], Y_sklearn[np.where(y == 'Iris-versicolor')[0], 1], 
            marker='^', color=colors[1], label='Iris-versicolor')
plt.scatter(Y_sklearn[np.where(y == 'Iris-virginica')[0], 0], Y_sklearn[np.where(y == 'Iris-virginica')[0], 1], 
            marker='o', color=colors[2], label='Iris-virginica')
plt.legend(loc='upper right')
plt.show(fig);

### Zadanie

Wylosuj po 10 cyfr ze zbioru cyfr i wykonaj na nim redukcję wymiarowość do 2. Narysuj położenie tych punktów tak jak zrobiliśmy to dla zbioru Iris.